In [2]:
import seaborn_image as isns
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import pylab as pl
from scipy.interpolate import griddata
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
n_pixels = 256
pixel_size = 55/1000 #mm
colormap = "rainbow"
magnification = 2.0
sns.set_style("darkgrid")

kernel_cold = np.ones((3,3))/(3*3)
kernel_hot = np.array([[0,0,0],[0,1,0],[0,0,0]]) # Identity

In [4]:
def padding(image,size):
    rows, columns = np.shape(image)[0], np.shape(image)[1]
    padded_image = np.zeros((rows + 2*size, columns + 2*size))
    #padded_image[size:size+rows, size:size+columns] = image
    padded_image[size:-size, size:-size] = image
    return padded_image

def deppading(padded_image,size):
    return padded_image[size:-size, size:-size]

def reduce_dim(imagen, factor):
    filas, columnas = np.shape(imagen)[0], np.shape(imagen)[1]
    new_image = np.zeros((int(filas/factor), int(columnas/factor)))

    for i in range(0,filas,factor):
        for j in range(0,columnas,factor):
            new_image[int(i/factor),int(j/factor)] = np.mean(imagen[i:i+factor, j:j+factor])
    return new_image

           
def preparacion(imagen):
    filas, columnas = np.shape(imagen)[0], np.shape(imagen)[1]

    promedio = np.mean(imagen)
    new_imagen = imagen

    criterio_filas, criterio_columnas = (2.0/3.0)*filas, (2.0/3.0)*columnas

    for i in range(filas):
        if( np.count_nonzero(imagen[i,:]) <  criterio_filas):
            new_imagen[i,:] = promedio*np.ones(filas)
    
    for j in range(columnas):
        if( np.count_nonzero(imagen[:,j]) < criterio_columnas ):
            new_imagen[:,j] = promedio*np.ones(columnas)
    return new_imagen


def interp(imagen):
    factor = 2

    x, y, z = [], [], [] # Creamos nuestros puntos "x", "y", "z"
    
    tamano_matriz = np.shape(imagen)
    filas, columnas = tamano_matriz[0], tamano_matriz[1]
    
    for i in range(filas):
        for j in range(columnas):
            x.append(i)
            y.append(j)
            z.append(imagen[i,j])
    
    x, y, z = np.array(x), np.array(y), np.array(z)
    
    # Aca ya tenemos nuestros datos x,y,z. Ahora solo queda interpolarlos.
    
    grid_x, grid_y = np.mgrid[0:filas-1:factor*filas*1j, 0:columnas-1:factor*columnas*1j] # Creamos la rejilla, tal como se hizo en el primer ejemplo de interpolación en 2D. En este caso exigimos que el nuevo tamaño de la imagen sea de 100x100 y se pone 0:9 dado que los valores de i y j varian desde 0 hasta 9 en principio.  
    
    #print("El tamaño de la matriz nueva será: ", np.shape(grid_x)) # Fijarse que el tamaño de la matriz si es el que queremos
    
    puntos_xy = np.vstack((x,y)).T # Hacemos este paso que ya se había explicado en el ejemplo previo
    
    imagen_nueva = griddata(puntos_xy, z, (grid_x, grid_y), method='cubic') # Hacemos interpolacion cúbica

    imagen_dim_reducida = reduce_dim(imagen_nueva, factor)

    return imagen_dim_reducida



def convolution(image,kernel_cold, kernel_hot, selective=False, stds_cold=1, stds_hot=1):
    excess_cold = int((np.shape(kernel_cold)[0] - 1)/2)
    excess_hot = int((np.shape(kernel_hot)[0] - 1)/2)

    n_pixels_kernel_cold = np.shape(kernel_cold)[0]*np.shape(kernel_cold)[1]
    acceptable_pixels_cold = ((np.shape(kernel_cold)[0]-1)/np.shape(kernel_cold)[0])*n_pixels_kernel_cold
    n_pixels_kernel_hot = np.shape(kernel_hot)[0]*np.shape(kernel_hot)[1]
    acceptable_pixels_hot = ((np.shape(kernel_hot)[0]-1)/np.shape(kernel_hot)[0])*n_pixels_kernel_hot

    mean_image, std_image = np.mean(image), np.std(image)

    minimum_acceptable = mean_image - stds_cold*std_image
    maximum_acceptable = mean_image + stds_hot*std_image
    print("minimum: " ,minimum_acceptable)
    print("maximum: " ,maximum_acceptable)

    if(selective==False):
        padded_image = padding(image,excess_cold)
        padded_rows, padded_columns = np.shape(padded_image)[0], np.shape(padded_image)[1]

        result = np.zeros( (padded_rows, padded_columns) )
        for i in range(excess_cold, padded_rows-excess_cold):
            for j in range(excess_cold, padded_columns-excess_cold):
                if(np.count_nonzero(padded_image[i-excess_cold:i+excess_cold+1, j-excess_cold:j+excess_cold+1]) > acceptable_pixels_cold):
                    multiplication = padded_image[i-excess_cold:i+excess_cold+1, j-excess_cold:j+excess_cold+1]*kernel_cold
                    result[i,j] = np.sum(multiplication)
                elif(np.count_nonzero(padded_image[i-excess_cold:i+excess_cold+1, j-excess_cold:j+excess_cold+1]) <= acceptable_pixels_cold):
                    imagen_interp = interp(padded_image[i-excess_cold:i+excess_cold+1, j-excess_cold:j+excess_cold+1])
                    multiplication = imagen_interp*kernel_cold
                    result[i,j] = np.sum(multiplication)
        result = deppading(result,excess_cold)
        return result

    if(selective==True):

        padded_image_cold = padding(image,excess_cold)
        padded_rows_cold, padded_columns_cold = np.shape(padded_image_cold)[0], np.shape(padded_image_cold)[1]

        result = padded_image_cold
        for i in range(excess_cold, padded_rows_cold-excess_cold):
            for j in range(excess_cold, padded_columns_cold-excess_cold):
                if(result[i,j] <= minimum_acceptable and np.count_nonzero(padded_image_cold[i-excess_cold:i+excess_cold+1, j-excess_cold:j+excess_cold+1]) > acceptable_pixels_cold):
                    multiplication = padded_image_cold[i-excess_cold:i+excess_cold+1, j-excess_cold:j+excess_cold+1]*kernel_cold
                    result[i,j] = np.sum(multiplication)
                elif(result[i,j] <= minimum_acceptable and np.count_nonzero(padded_image_cold[i-excess_cold:i+excess_cold+1, j-excess_cold:j+excess_cold+1]) <= acceptable_pixels_cold):
                    imagen_interp = interp(padded_image_cold[i-excess_cold:i+excess_cold+1, j-excess_cold:j+excess_cold+1])
                    multiplication = imagen_interp*kernel_cold
                    result[i,j] = np.sum(multiplication)

        result = deppading(result,excess_cold)

        padded_image_hot = padding(result,excess_hot)
        padded_rows_hot, padded_columns_hot = np.shape(padded_image_hot)[0], np.shape(padded_image_hot)[1]

        result2 = padded_image_hot

        for i in range(excess_hot, padded_rows_hot-excess_hot):
            for j in range(excess_hot, padded_columns_hot-excess_hot):
                if(result2[i,j] >= maximum_acceptable and np.count_nonzero(padded_image_hot[i-excess_hot:i+excess_hot+1, j-excess_hot:j+excess_hot+1]) > acceptable_pixels_hot):
                    multiplication = padded_image_hot[i-excess_hot:i+excess_hot+1, j-excess_hot:j+excess_hot+1]*kernel_hot
                    result2[i,j] = np.sum(multiplication)
                elif(result2[i,j] >= maximum_acceptable and np.count_nonzero(padded_image_hot[i-excess_hot:i+excess_hot+1, j-excess_hot:j+excess_hot+1]) <= acceptable_pixels_hot):
                    imagen_interp = interp(padded_image_hot[i-excess_hot:i+excess_hot+1, j-excess_hot:j+excess_hot+1])
                    multiplication = imagen_interp*kernel_hot
                    result2[i,j] = np.sum(multiplication)

        result2 = deppading(result2,excess_hot)
        return result2

def histo_colormap(histo,cmap, inf=0, sup=0):
    if(inf==0 and sup==0):
        colormap_values = plt.get_cmap(cmap, len(histo.patches))
        for i in range(len(histo.patches)):
            histo.patches[i].set_facecolor(colormap_values(i))
    else:
        indxs = []
        for i,patch in enumerate(histo.patches):
            if(patch.get_x() >= inf and patch.get_x() <= sup):
                indxs.append(i)

        colormap_values = plt.get_cmap(cmap, len(indxs))
        for i in indxs:
            histo.patches[i].set_facecolor(colormap_values(i))

def plots(imagen_entry, val_min, val_max, std_cold=1000, std_hot=1000, kernel_c = kernel_cold, kernel_h= kernel_hot):
    imagen = imagen_entry

    print("number of zeros: ", np.count_nonzero(imagen==0)) # Count zeros

    imagen = convolution(imagen, kernel_c, kernel_h, selective=True, stds_cold=std_cold, stds_hot=std_hot)

    plt.figure(figsize=(10,5))

    f = sns.histplot(x=np.ravel(imagen), bins=100, cbar=True, ec="black")
    histo_colormap(f,colormap, inf=val_min, sup=val_max)
    plt.xlim(xmin =val_min, xmax=val_max)

    print("number of zeros: ", np.count_nonzero(imagen==0)) # Count zeros
    
    k = isns.imghist(imagen, cmap=colormap, dx=pixel_size/magnification, units=r"mm", orientation="v", bins=100, vmax=val_max, vmin=val_min)
    isns.set_scalebar(length_fraction=0.1)

    return f, k


In [5]:
#FF = np.genfromtxt("FF/FF_PMMA_28kV.txt")
RAWS_data = np.genfromtxt("1. RAW/RAW.txt")

n_RAWS = 350
n_FFS = 100
offset = 35

RAWS = []
FFS = []

for ind in range(0,n_pixels*n_RAWS,n_pixels):
    RAWS.append(np.rot90(RAWS_data[ind:ind+n_pixels],1)[offset:,:])

RAWS = np.array(RAWS)

for ind in range(n_FFS):
    FF = np.genfromtxt( "2. FF/FF_{0}.txt".format(str(ind).zfill(2)) )
    FFS.append(np.rot90(FF,1)[offset:,:])

In [6]:
FFS = np.array(FFS)
FF = np.sum(FFS, axis=0) #suma de todas las imagenes, no de todos los elementos individuales

In [7]:
imagen = RAWS[0]
max_image = 1500#np.amax(np.ravel(imagen))
min_image = np.amin(np.ravel(imagen))
interact(plots, imagen_entry = fixed(imagen), val_min = fixed(min_image), val_max=widgets.IntSlider( min=min_image, max=max_image, step=1, value=max_image ), std_cold= fixed(1.7), std_hot= fixed(2.7),
kernel_c = fixed(kernel_cold), kernel_h = fixed(kernel_hot))

interactive(children=(IntSlider(value=1500, description='val_max', max=1500), Output()), _dom_classes=('widget…

<function __main__.plots(imagen_entry, val_min, val_max, std_cold=1000, std_hot=1000, kernel_c=array([[0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111]]), kernel_h=array([[0, 0, 0],
       [0, 1, 0],
       [0, 0, 0]]))>

In [8]:
imagen = FF
max_image = np.amax(np.ravel(imagen))
min_image = np.amin(np.ravel(imagen))
interact(plots, imagen_entry = fixed(imagen), val_min = fixed(min_image), val_max=widgets.IntSlider( min=min_image, max=max_image, step=1, value=max_image ), std_cold= fixed(0.5), std_hot= fixed(1000),
kernel_c = fixed(kernel_cold), kernel_h = fixed(kernel_hot))

interactive(children=(IntSlider(value=328370, description='val_max', max=328370), Output()), _dom_classes=('wi…

<function __main__.plots(imagen_entry, val_min, val_max, std_cold=1000, std_hot=1000, kernel_c=array([[0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111]]), kernel_h=array([[0, 0, 0],
       [0, 1, 0],
       [0, 0, 0]]))>

In [9]:
RAW_corr = convolution(RAWS[0], kernel_cold, kernel_hot, selective=True, stds_cold=1.7, stds_hot=1000)
FF_corr = convolution(FF, kernel_cold, kernel_hot, selective=True, stds_cold=0.5, stds_hot=1000)

imagen = np.log(FF_corr/RAW_corr)
max_image = np.amax(np.ravel(imagen))
min_image = np.amin(np.ravel(imagen))
interact(plots, imagen_entry = fixed(imagen), val_min = fixed(min_image), val_max=widgets.IntSlider( min=min_image, max=max_image, step=1, value=max_image ), std_cold= fixed(1000), std_hot= fixed(1000), 
kernel_c = fixed(kernel_cold), kernel_h = fixed(kernel_hot))

minimum:  3026.3728612941604
maximum:  2597626.2567636413
minimum:  240829.1808271446
maximum:  17776124.280859992


interactive(children=(IntSlider(value=9, description='val_max', max=9, min=2), Output()), _dom_classes=('widge…

<function __main__.plots(imagen_entry, val_min, val_max, std_cold=1000, std_hot=1000, kernel_c=array([[0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111]]), kernel_h=array([[0, 0, 0],
       [0, 1, 0],
       [0, 0, 0]]))>

In [10]:
%%capture
magnification = 2.0
FF_corr = convolution(FF, kernel_cold, kernel_hot, selective=True, stds_cold=0.5, stds_hot=1000)

for i, RAW in enumerate(RAWS):
    RAW_corr = convolution(RAW, kernel_cold, kernel_hot, selective=True, stds_cold=1.7, stds_hot=1000)
    CORR = np.log(FF_corr/RAW_corr)
    isns.imghist(CORR, cmap=colormap, dx=pixel_size/magnification, units=r"mm", orientation="v", bins=100, vmin=min_image, vmax=max_image)
    isns.set_scalebar(length_fraction=0.1)
    plt.savefig("CORR/CORR_{0}.png".format(i))
    plt.clf()
    np.savetxt("CORR_txt/CORR_{0}.txt".format(i), CORR)
    

In [11]:
print("Corr min and max: ", min_image, max_image)

Corr min and max:  2.998415264123847 9.83390410406601
